In [9]:
from noworkflow.now.persistence.models import Evaluation, Activation
from noworkflow.now.models.dependency_querier import DependencyQuerier
from noworkflow.now.models.dependency_querier.node_context import NodeContext
from noworkflow.now.models.dependency_querier.querier_options import QuerierOptions
from noworkflow.now.persistence.models import Trial
from copy import copy

import ipdb
import numpy as np
from pprint import pprint

In [10]:
import numpy as np

# Helper functions
def get_activation(activation_id, trial_id = None):
    if not trial_id:
        trial_id =  __noworkflow__.trial_id
    return Activation((trial_id, activation_id))

def get_evaluation(evaluation_id, trial_id = None):
    if not trial_id:
        trial_id =  __noworkflow__.trial_id
    return Evaluation((trial_id, evaluation_id))

def get_act_evals(activation_id):
    activation = get_activation(activation_id)
    return activation.evaluations    

def print_act_forward(activation_id=1):
    activation = get_activation(activation_id)
    
    for i in list(activation.dependent_dependencies):
        pprint.pprint(i.to_dict())
        
def print_act_backward(activation_id=1):
    activation = get_activation(activation_id)
    
    for i in list(activation.dependency_dependencies):
        pprint.pprint(i.to_dict())

def print_act_evals(activation_id=1):
    activation = get_activation(activation_id)

    for item in activation.evaluations:
        pprint.pprint(item.to_dict())
        


### Test case

Here starts my (only) test case. The flow goes through cells defining module, mae and rmse, variable definitions, variable manipulation and _var_final_ being our target variable, being dependent of its previous cells.


In [11]:
def module(num): 
    if num >= 0:
        return int(num)
    else:
        return int(-1*num)

def mae(x, y):
    return np.mean(np.abs((x - y)))

def rmse(x, y):
    return np.sqrt(np.mean((x - y)**2))

def stress_func(m, n):
    temp = m * n
    x = 10
    y = 100
    
    temp = np.random.rand()*temp
    
    final = np.max([int(x+y), int(temp)])
    
    return final

In [12]:
var_a = 10

In [13]:
var_b = -100

In [14]:
x = 50
y = 30

In [15]:
rme_err = mae(x, var_b)

In [16]:
var_c = stress_func(x, y)

In [17]:
var_d = var_c * rme_err

In [18]:
var_e = mae(var_c, var_b)
x = var_e + 10

In [19]:
#k = module(rmse_err)
var_final = now_variable('var_final', rmse(var_c, x))

Evaluation(id=180, checkpoint=186.702766308, code_component_id=1083, activation_id=174, repr=110.0)


### New functions test

In [7]:
evals = get_evaluation(131)

AttributeError: 'NoneType' object has no attribute 'trial_id'

In [27]:
evals


evaluation(9236ebae-a8e2-4b65-9222-02f7a9454868, 131, 6.446164974, 591, 124, '110.0', 144).

### Outputing the dependencies of a variable

print_precedents() trying to output a cleaner output from get_precedent() results

In [61]:
class NotebookQuerierOptions(QuerierOptions):
    global body_function_def
    dep_list = []
    
    
    def __init__(self, level, *args, **kwargs):
        QuerierOptions.__init__(self, *args, **kwargs) # change it to super when integrating in the main code
        self.level = level
    
    def visit_arrow(self, context, neighbor):
                
        # keeping 
        if neighbor.evaluation.code_component.type == 'function_def':
            body_function_def.append(int(neighbor.evaluation.code_component.id))
       
        arrow_list = ('argument', '<.__class__>', 'item')
        type_list = ('add', 'literal', 'mult', 'div', 'param', 'sub', 'attribute', 'usub', 'function_def')
        
        context_code_comp = context.evaluation.code_component
        neighbor_code_comp = neighbor.evaluation.code_component
           
        if neighbor.arrow not in arrow_list:
            if context_code_comp.type not in type_list:
                if neighbor_code_comp.type not in type_list:
                    if not (neighbor.arrow == 'use' and context_code_comp.type == 'call'):
                        if (neighbor_code_comp.container_id != None):
                            if (neighbor_code_comp.container_id not in body_function_def) or self.level:
                                self.dep_list.append((str(context.evaluation.checkpoint), str(context.evaluation.id), context_code_comp.name, context.evaluation.repr))

    def public_print(self):
        global dep_dict
        
        dep_dict = {i[0] : i[1] for i in enumerate(self.dep_list)}
        return dep_dict
        
                                
body_function_def = []
dep_dict = {}

nbOptions = NotebookQuerierOptions(level=False)
querier = DependencyQuerier(options=nbOptions)
nodes_to_visit, visited, found = querier.navigate_dependencies([evals])  



In [62]:
nbOptions.public_print()

{0: ('6.446164974', '131', 'rmse(var_c, x)', '110.0'),
 1: ('6.4454572290000005', '130', 'x', '1257.0'),
 2: ('6.328109044000001', '120', 'var_e', '1247.0'),
 3: ('6.328066595000001', '119', 'var_e', '1247.0'),
 4: ('6.328048174000001', '107', 'mae(var_c, var_b)', '1247.0'),
 5: ('6.327266418000001', '106', 'var_b', '-100'),
 6: ('6.327235152', '105', 'var_c', '1147'),
 7: ('6.0874868069999994', '97', 'var_c', '1147'),
 8: ('6.0874656510000005', '61', 'stress_func(x, y)', '1147'),
 9: ('6.086186027', '60', 'y', '30'),
 10: ('6.0861170609999995', '59', 'x', '50')}

In [52]:
from noworkflow.now.persistence.models.stage_tags import *
from noworkflow.now.persistence import relational
from noworkflow.now.persistence.models.base import proxy_gen

In [47]:
__noworkflow__.trial_id

'9236ebae-a8e2-4b65-9222-02f7a9454868'

In [53]:
list(proxy_gen(relational.session.query(StageTags.m).filter(StageTags.m.trial_id == __noworkflow__.trial_id)))

[access(9236ebae-a8e2-4b65-9222-02f7a9454868, f1, 'var_final', '110.0', nil, nil, nil).,
 access(9236ebae-a8e2-4b65-9222-02f7a9454868, f2, 'var_final', '110.0', nil, nil, nil).]

In [64]:
a = list(proxy_gen(relational.session.query(StageTags.m).filter(StageTags.m.trial_id == __noworkflow__.trial_id)))

In [73]:
for i in a:
    print(i.name)

var_final
var_final


In [77]:
i.name

'var_final'

In [96]:
    from noworkflow.now.persistence.models.stage_tags import *
    from noworkflow.now.persistence import relational
    from noworkflow.now.persistence.models.base import proxy_gen
    
    def get_pre_tagged(glanularity = False):


        dict_tags = []

        tagged_values = list(proxy_gen(relational.session.query(StageTags.m).filter(StageTags.m.trial_id == __noworkflow__.trial_id)))
        
        for var in tagged_values:
            dict_tags[var.name] = get_pre(var.name)
            
        return dict_tags

In [84]:
tagged_values = list(proxy_gen(relational.session.query(StageTags.m).filter(StageTags.m.trial_id == __noworkflow__.trial_id)))

In [103]:
dict_tags = []
for var in tagged_values:
    dict_tags.append(get_pre(var.name))

In [108]:
dict_tags[0]

{0: ('27.472266372', '3778', 'rmse(var_c, x)', '110.0'),
 1: ('27.471551243', '3777', 'x', '346.0'),
 2: ('27.272975153', '3768', 'var_e', '336.0'),
 3: ('27.272933288', '3767', 'var_e', '336.0'),
 4: ('27.272916228', '3755', 'mae(var_c, var_b)', '336.0'),
 5: ('27.272296989', '3754', 'var_b', '-100'),
 6: ('27.272264149', '3753', 'var_c', '236'),
 7: ('26.880302824', '3745', 'var_c', '236'),
 8: ('26.880281984', '3713', 'stress_func(x, y)', '236'),
 9: ('26.878983443', '3712', 'y', '30'),
 10: ('26.878857644', '3711', 'x', '50'),
 11: ('27.472266372', '3778', 'rmse(var_c, x)', '110.0'),
 12: ('27.471551243', '3777', 'x', '346.0'),
 13: ('27.272975153', '3768', 'var_e', '336.0'),
 14: ('27.272933288', '3767', 'var_e', '336.0'),
 15: ('27.272916228', '3755', 'mae(var_c, var_b)', '336.0'),
 16: ('27.272296989', '3754', 'var_b', '-100'),
 17: ('27.272264149', '3753', 'var_c', '236'),
 18: ('26.880302824', '3745', 'var_c', '236'),
 19: ('26.880281984', '3713', 'stress_func(x, y)', '236'),


In [109]:
dict_tags[1]

{0: ('27.472266372', '3778', 'rmse(var_c, x)', '110.0'),
 1: ('27.471551243', '3777', 'x', '346.0'),
 2: ('27.272975153', '3768', 'var_e', '336.0'),
 3: ('27.272933288', '3767', 'var_e', '336.0'),
 4: ('27.272916228', '3755', 'mae(var_c, var_b)', '336.0'),
 5: ('27.272296989', '3754', 'var_b', '-100'),
 6: ('27.272264149', '3753', 'var_c', '236'),
 7: ('26.880302824', '3745', 'var_c', '236'),
 8: ('26.880281984', '3713', 'stress_func(x, y)', '236'),
 9: ('26.878983443', '3712', 'y', '30'),
 10: ('26.878857644', '3711', 'x', '50'),
 11: ('27.472266372', '3778', 'rmse(var_c, x)', '110.0'),
 12: ('27.471551243', '3777', 'x', '346.0'),
 13: ('27.272975153', '3768', 'var_e', '336.0'),
 14: ('27.272933288', '3767', 'var_e', '336.0'),
 15: ('27.272916228', '3755', 'mae(var_c, var_b)', '336.0'),
 16: ('27.272296989', '3754', 'var_b', '-100'),
 17: ('27.272264149', '3753', 'var_c', '236'),
 18: ('26.880302824', '3745', 'var_c', '236'),
 19: ('26.880281984', '3713', 'stress_func(x, y)', '236'),
